In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import length,trim,col,avg,year,to_date,udf,min,max
from pyspark.sql.types import DateType
spark = SparkSession.builder.getOrCreate()
#Data Frame Creation
df=spark.read.json(r".\data\amazon.json")
#creating a dataframe which will collect data of avg of ratings 
grp_df=df.withColumn("review_length",length(trim(col("reviewText")))).groupBy('asin').agg(avg("overall").alias("rating_avg"),max("review_length").alias("review_len_avg"))
#Shows Output of Items with minimum rating
grp_df.filter((grp_df.rating_avg == grp_df.agg({"rating_avg":"min"}).collect()[0][0])).show()
#Shows Output of items with Max rating
grp_df.filter((grp_df.rating_avg == grp_df.agg({"rating_avg":"max"}).collect()[0][0])).show()
#shows output of item with max length of review
grp_df.filter((grp_df.review_len_avg == grp_df.agg({"review_len_avg":"max"}).collect()[0][0])).show()
#Shows Dataframe with reviewTime changed to Date format
df=df.withColumn('reviewTime',to_date('reviewTime',"MM d, yyyy"))
df.show()
df.write.parquet("output/Assignment1/",mode='overwrite')

#User Defined Function which will return a column with value having details about verified user rating or not
@udf(returnType=StringType()) 
def valid_rating_info(input1,input2):
    if input1 :
        return 'verified user ' + str(input2)
    else:
        return 'Unverified User '+ str(input2)

df.withColumn('User_status',valid_rating_info("verified","reviewerID")).show()
